# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Ejemplos de Aprendizaje Automático (Machine Learning): K-means** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [ ]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MLSpark-K-means") \
    .master("spark://078b2e28e517:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

# Preparación de Datos

In [ ]:
from team_name.spark_utils import SparkUtils

# Sample data in Python (e.g., 2D points)
data = [
    (0, 1.0, 1.0),
    (1, 2.0, 1.0),
    (2, 4.0, 5.0),
    (3, 5.0, 5.0),
    (4, 10.0, 10.0),
    (5, 12.0, 11.0)
]

# Define schema for the DataFrame
schema = SparkUtils.generate_schema([("id", "integer"), ("x", "float"), ("y", "float")])

# Create DataFrame for k means
random_points_df = spark.createDataFrame(data, schema)

### Assemble the features into a single vector column

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["x", "y"], outputCol="features")
assembled_df = assembler.transform(random_points_df)

# Initialize KMeans

In [ ]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans().setK(2).setSeed(19)

# TRAINNING

In [ ]:
model = kmeans.fit(assembled_df)

# PREDICTIONS

In [ ]:
predictions = model.transform(assembled_df)

# EVALUATE MODEL

In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print(f"Silhouette score: {silhouette}")

# Show the result
print("Cluster Centers: ")
for center in model.clusterCenters():
    print(center)

In [ ]:
sc.stop()